In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange


from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


#
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


#
from tsnecuda import TSNE

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds


In [3]:
########################################
#### COMPUTE COS SIMILARITY METRICS ####
########################################

fname = '/media/cat/4TB/donato/DON-006084/20210519/suite2p/plane0/binarized_traces.npz'
data = np.load(fname,
              allow_pickle=True)

#
events = data['spikes'].T
print (events.shape)

#
A_sparse = sparse.csr_matrix(events)

similarities = cosine_similarity(A_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

#also can output sparse matrices
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

print (similarities_sparse.shape)

matrix_dense =  scipy.sparse.csr_matrix.todense(similarities_sparse)


(55740, 531)
pairwise dense output:
 [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.99999976 0.0793258  ... 0.20295794 0.17623338 0.14517485]
 [0.         0.0793258  1.0000001  ... 0.06003026 0.11673368 0.12455288]
 ...
 [0.         0.20295794 0.06003026 ... 0.99999994 0.10317299 0.04908124]
 [0.         0.17623338 0.11673368 ... 0.10317299 1.0000001  0.07890832]
 [0.         0.14517485 0.12455288 ... 0.04908124 0.07890832 1.        ]]

pairwise sparse output:
   (1, 53331)	0.010470537
  (1, 52476)	0.011776504
  (1, 52431)	0.012535518
  (1, 51509)	0.011023164
  (1, 49217)	0.008274922
  (1, 48519)	0.012091629
  (1, 47053)	0.006607328
  (1, 42752)	0.008718313
  (1, 41734)	0.014276251
  (1, 40945)	0.011398115
  (1, 40583)	0.011994401
  (1, 39847)	0.009724477
  (1, 39628)	0.013658968
  (1, 37634)	0.025490155
  (1, 36478)	0.01970323
  (1, 36133)	0.024191786
  (1, 35713)	0.0129825035
  (1, 35158)	0.016832167
  (1, 31079)	0.008263586
  (1, 31017)	0.012

In [4]:
# matrix_dense =  scipy.sparse.csr_matrix.todense(similarities_sparse)

#
down_sample = 10
img2 = []
for k in trange(0,matrix_dense.shape[0],down_sample):
    temp = matrix_dense[k:k+down_sample]
    temp = np.sum(temp, axis=0)
    #print (temp.shape)
    img2.append(temp)

img = np.array(img2).squeeze().T
print (img.shape)

img2 = []
for k in trange(0,img.shape[0],down_sample):
    temp = img[k:k+down_sample]
    temp = np.sum(temp, axis=0)
    #print (temp.shape)
    img2.append(temp)

img = np.array(img2).squeeze().T
print (img.shape)

plt.imshow(img)
plt.show()

100%|██████████████████████████████████████████████████████████████████████████████████| 5574/5574 [00:01<00:00, 2884.48it/s]


(55740, 5574)


100%|██████████████████████████████████████████████████████████████████████████████████| 5574/5574 [00:01<00:00, 5532.77it/s]


(5574, 5574)


In [7]:
# 
def run_UMAP(data, 
             n_neighbors=50,
             min_dist=0.1,
             n_components=3,
             metric='euclidean'):
    
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric
    )
    
    u = fit.fit_transform(data)
    
    return u

X_u = run_UMAP(img)

print (X_u.shape)

(5574, 3)


In [5]:
pca = PCA(n_components=100)
X_pca = pca.fit_transform(img)
print (X_pca.shape)

(5574, 100)


In [8]:
v = visualize.Visualize()
times = np.arange(X_pca.shape[0])
fig = v.plot_scatter_3d( 
               u[:,:3], 
               times, 
                'times',
    'magma',
               1,1
              )
fig.show()

In [24]:
v = visualize.Visualize()
times = np.arange(X_pca.shape[0])
fig = v.plot_scatter_3d( 
               X_pca[:,:3], 
               times, 
                'times',
    'magma',
               1,1
              )
fig.show()